In [2]:
import re
import os
import requests
import zipfile
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
import arff

In [3]:
root_url = 'http://www.kdnuggets.com/data_mining_course/data/'
root_data = 'data'
data_file = 'final_project_data.zip'
train_data_class = 'pp5i_train_class.txt'
train_data_file = 'pp5i_train.gr.csv'
test_data_file = 'pp5i_test.gr.csv'
norm_train_file = 'pp5i.train.norm.tmp'
norm_test_file = 'pp5i.test.norm.tmp'
top_t_value_file = 'pp5i_train.top'

In [4]:
def unzip_file(file_addr,output_dir,remove=False) :
    with zipfile.ZipFile(file_addr,"r") as zip_ref:
        zip_ref.extractall(output_dir)
    if remove :  
        os.remove(file_addr)
    return os.listdir(output_dir)

def download_file(url,output_file,unzip=False,output_dir='') :
    if not os.path.exists(os.path.join(output_dir,output_file)) :
        # Streaming, so we can iterate over the response.
        r = requests.get(url, stream=True)

        # Total size in bytes.
        total_size = int(r.headers.get('content-length', 0)); 
        block_size = 1024
        wrote = 0 
        with open(os.path.join(output_dir,output_file), 'wb') as f:
            for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='KB', unit_scale=True):
                wrote = wrote  + len(data)
                f.write(data)
        if total_size != 0 and wrote != total_size:
            print("ERROR, something went wrong")
    if unzip :
        return unzip_file(os.path.join(output_dir,output_file),output_dir,True)
    return os.path.join(output_dir,output_file)

file = download_file(root_url + data_file,data_file,True,root_data)

933KB [00:17, 53.9KB/s]                          


In [5]:
train_file = './data/' + train_data_file
train_data = pd.read_csv(train_file)
test_file = './data/' + test_data_file
test_data = pd.read_csv(test_file)
train_data.head(5)

,SNO,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,A28102_at,30,46,31,31,26,28,35,29,21,...,35,49,31,51,71,68,77,56,41,38
1,AB000114_at,22,31,19,16,26,24,29,20,23,...,38,30,22,24,19,21,22,25,21,17
2,AB000115_at,29,70,12,11,14,13,14,18,10,...,20,205,16,61,62,35,30,65,32,25
3,AB000220_at,76,208,244,39,85,23,634,159,50,...,55,203,152,53,28,30,31,31,27,16
4,AB000409_at,167,211,179,119,161,166,228,267,260,...,137,180,107,147,170,131,132,158,164,172


## Step 1 . Data Cleaning
Threshold both train and test data to a minimum value of 20, maximum of 16,000

In [6]:
def normalize_data(data,save_path) :
    def normalize_row(x) :
        if type(x) == str or (x > 20 and x < 16000) :
            return x
        else :
            if np.abs(x - 16000) > np.abs(x - 20) :
                return 20
            else :
                return 16000
    data = data.applymap(normalize_row)
    data.to_csv(save_path,index=False)
normalize_data(train_data,os.path.join(root_data,norm_train_file))
normalize_data(test_data,os.path.join(root_data,norm_test_file))
norm_train_data = pd.read_csv(os.path.join(root_data,norm_train_file))

## Step 2. Selecting top genes by class
- remove from train data genes with fold differences across samples less than 2
- for each class, generate subsets with top 2,4,6,8,10,12,15,20,25, and 30 top genes with the highest T-value (Optional: for each class, select top genes using highest absolute T-value (i.e. also include genes with high negative T-value)
- for each N=2,4,6,8,10,12,15,20,25,30 combine top genes for each class into one file (removing duplicates, if any) and call the resulting file pp5i_train.topN.gr.csv
- Add the class as the last column, remove sample no, transpose each file to "genes-in-columns" format and convert it to arff.

In [7]:
norm_train_data['fold variation'] = norm_train_data.apply(lambda row: max(row[1:])/min(row[1:]),axis=1)
norm_train_data = norm_train_data[norm_train_data['fold variation'] > 2]
norm_train_data

,SNO,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,fold variation
0,A28102_at,30,46,31,31,26,28,35,29,21,...,49,31,51,71,68,77,56,41,38,5.600000
1,AB000114_at,22,31,20,20,26,24,29,20,23,...,30,22,24,20,21,22,25,21,20,2.950000
2,AB000115_at,29,70,20,20,20,20,20,20,20,...,205,20,61,62,35,30,65,32,25,10.950000
3,AB000220_at,76,208,244,39,85,23,634,159,50,...,203,152,53,28,30,31,31,27,20,31.700000
4,AB000409_at,167,211,179,119,161,166,228,267,260,...,180,107,147,170,131,132,158,164,172,7.897436
5,AB000449_at,32,20,229,20,34,43,56,20,20,...,25,20,20,20,20,20,20,20,20,11.450000
6,AB000450_at,20,20,20,20,20,20,21,20,20,...,20,20,20,21,22,20,27,20,20,2.100000
7,AB000460_at,165,184,219,156,206,197,249,138,147,...,184,140,148,154,140,150,173,165,145,3.270073
8,AB000462_at,47,36,41,26,34,40,45,133,40,...,26,55,54,57,37,43,54,74,48,11.000000
9,AB000464_at,101,130,131,86,140,113,121,250,264,...,115,103,105,117,81,128,108,121,118,4.661972


T-value is defined as (Avg1 - Avg2) / sqrt(Stdev1*Stdev/N1 + Stdev2*Stdev2/N2)

In [40]:
listone = []
listtwo = [4,5,6]

mergedlist = listone + listtwo
mergedlist

[4, 5, 6]

In [51]:
def t_value(row,start_c,end_c,first,end) :
    avg = []
    std = []
    n = []
    class1=row[start_c:end_c]
    class2=pd.concat([row[first:start_c],row[end_c:end]])
    for i,attr in enumerate((class1,class2)) :
        n.append(len(attr))
        avg.append(np.mean(attr))
        sum_val = np.sum(attr)
        sum_sq = np.sum(np.square(attr))
        std.append(np.sqrt((n[i]*sum_sq - sum_val*sum_val)/(n[i]*(n[i]-1))))
    row['t_value'] = (avg[0] - avg[1]) / np.sqrt(std[0]*std[0]/n[0] + std[1]*std[1]/n[1])
    return row
ranges = [[1,40],[40,47],[47,54],[54,64],[64,70]]
p_data = []
for i in range(len(ranges)):
    p_data.append(norm_train_data.iloc[:,:].apply(lambda row:t_value(row,ranges[i][0],ranges[i][1],1,70),axis=1))

In [52]:
p_data[0]

,SNO,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,fold variation,t_value
0,A28102_at,30,46,31,31,26,28,35,29,21,...,31,51,71,68,77,56,41,38,5.600000,-4.554519
1,AB000114_at,22,31,20,20,26,24,29,20,23,...,22,24,20,21,22,25,21,20,2.950000,0.782452
2,AB000115_at,29,70,20,20,20,20,20,20,20,...,20,61,62,35,30,65,32,25,10.950000,-2.898405
3,AB000220_at,76,208,244,39,85,23,634,159,50,...,152,53,28,30,31,31,27,20,31.700000,0.301633
4,AB000409_at,167,211,179,119,161,166,228,267,260,...,107,147,170,131,132,158,164,172,7.897436,1.349352
5,AB000449_at,32,20,229,20,34,43,56,20,20,...,20,20,20,20,20,20,20,20,11.450000,1.023761
6,AB000450_at,20,20,20,20,20,20,21,20,20,...,20,20,21,22,20,27,20,20,2.100000,-1.507670
7,AB000460_at,165,184,219,156,206,197,249,138,147,...,140,148,154,140,150,173,165,145,3.270073,1.018844
8,AB000462_at,47,36,41,26,34,40,45,133,40,...,55,54,57,37,43,54,74,48,11.000000,0.677845
9,AB000464_at,101,130,131,86,140,113,121,250,264,...,103,105,117,81,128,108,121,118,4.661972,1.570427


In [56]:
t_value_largest = p_data[0].nlargest(30,['t_value'])[['SNO','t_value']]
t_value_largest

,SNO,t_value
2773,U16954_at,11.672150
1868,M31303_rna1_at,11.102608
3704,U79262_at,10.396512
528,D80004_at,10.210026
404,D50310_at,9.780443
5132,Z69915_at,9.679137
2305,M93119_at,9.670301
6411,U09087_s_at,9.055068
6402,X73358_s_at,8.645249
2954,U30521_at,8.464778


In [102]:
def save_nlargest(number,input_data,path):
    frames = []
    for i in range(len(input_data)):
        frames.append(input_data[i].nlargest(number,['t_value']))
    result = pd.concat(frames).iloc[:,:-2].drop_duplicates().reset_index(drop=True)
    result.transpose().to_csv(path+str(number)+'.gr.csv',index=False)
path = os.path.join(root_data,top_t_value_file)
numbers = [2,4,6,8,10,12,15,20,25,30]
for i in range(len(numbers)):   
    save_nlargest(numbers[i],p_data,path)

In [103]:
def addclass(input):
    return input.reindex_axis(sorted(input.columns), axis=1).assign(Class=pd.read_csv(os.path.join(root_data,train_data_class)).Class)
tops = []
for i in range(len(numbers)):   
    tops.append(addclass(pd.read_csv(path+str(numbers[i])+'.gr.csv',header=1)))
tops[9]

,AJ001421_at,D00763_at,D13631_s_at,D14043_at,D14662_at,D17716_at,D25304_at,D29675_s_at,D31767_at,D31890_at,...,X96401_at,X98085_at,X99720_rna1_at,Y07604_at,Z19554_s_at,Z29064_at,Z56281_at,Z69915_at,Z97074_at,Class
0,79,491,21,165,243,116,270,107,260,332,...,20,505,143,193,1086,82,40,160,51,MED
1,104,350,54,166,228,117,250,47,290,291,...,39,327,157,80,1190,82,44,172,66,MED
2,98,378,20,100,185,99,140,36,415,272,...,104,241,208,143,965,41,20,184,66,MED
3,94,476,31,117,238,84,167,76,329,330,...,66,299,185,199,1225,104,43,164,46,MED
4,70,326,20,142,173,112,141,42,313,193,...,132,260,140,109,587,32,20,157,78,MED
5,123,337,20,100,138,114,120,46,393,262,...,104,271,168,109,379,51,20,158,61,MED
6,68,311,20,27,279,67,110,25,321,284,...,131,211,202,65,302,62,20,186,81,MED
7,37,151,20,61,52,268,77,84,20,216,...,187,233,28,189,1158,51,20,184,33,MED
8,22,65,20,49,52,136,56,71,160,226,...,341,149,30,116,1639,55,20,124,73,MED
9,104,171,20,24,113,243,109,62,146,288,...,20,303,34,446,2225,44,20,159,39,MED


In [104]:
# have problem should add @attribute Class {MED,MGL,RHB,EPD,JPA} manually
def save_arff(path,values,relation,names):
    arff.dump(path,values, relation=relation, names=names)
for i in range(len(numbers)):   
    save_arff(path+str(numbers[i])+'.gr.arff',tops[i].values,"default",tops[i].columns)

## Step 3. Find the best classifier/best gene set combination
- NaiveBayes
- J48
- IB1
- IBk (for each value of K=2, 3, 4)
- one more Weka classifier of your choice -- that can work with multiclass data.

### NaiveBayes 
- N = 2 ![](./images/nb_2.png)
- N = 4 ![](./images/nb_4.png)
- N = 6 ![](./images/nb_6.png)
- N = 8 ![](./images/nb_8.png)
- N = 10 ![](./images/nb_10.png)
- N = 12 ![](./images/nb_12.png)
- N = 15 ![](./images/nb_15.png)
- N = 20 ![](./images/nb_20.png)
- N = 25 ![](./images/nb_25.png)
- N = 30 ![](./images/nb_30.png)

### J48
- N = 2 ![](./images/j48_2.png)
- N = 4 ![](./images/j48_4.png)
- N = 6 ![](./images/j48_6.png)
- N = 8 ![](./images/j48_8.png)
- N = 10 ![](./images/j48_10.png)
- N = 12 ![](./images/j48_12.png)
- N = 15 ![](./images/j48_15.png)
- N = 20 ![](./images/j48_20.png)
- N = 25 ![](./images/j48_25.png)
- N = 30 ![](./images/j48_30.png)

### IB1
- N = 2 ![](./images/ib1_2.png)
- N = 4 ![](./images/ib1_4.png)
- N = 6 ![](./images/ib1_6.png)
- N = 8 ![](./images/ib1_8.png)
- N = 10 ![](./images/ib1_10.png)
- N = 12 ![](./images/ib1_12.png)
- N = 15 ![](./images/ib1_15.png)
- N = 20 ![](./images/ib1_20.png)
- N = 25 ![](./images/ib1_25.png)
- N = 30 ![](./images/ib1_30.png)

### IBK 
- ib2 for N = 12 ![](./images/ib2_12.png)
- ib3 for N = 12 ![](./images/ib3_12.png)
- ib4 for N = 12 ![](./images/ib4_12.png)

** corresponding this result , ib1 is best classifier on training set and best gene set for N = 25 **

## Step 4. Generate predictions for the test set

In [105]:
test_data = pd.read_csv(test_file)
a = tops[8].columns.values
test_data = test_data[test_data['SNO'].isin(a)].transpose()
test_data.to_csv('./data/pp5i_test.bestN.csv',index=False)
test_data = pd.read_csv('./data/pp5i_test.bestN.csv',header = 1)
test_data = test_data.reindex_axis(sorted(test_data.columns), axis=1).assign(Class="?")
test_data

,D00763_at,D13631_s_at,D14043_at,D14662_at,D17716_at,D25304_at,D29675_s_at,D31767_at,D31890_at,D42041_at,...,X76105_at,X78706_at,X84195_at,X96401_at,Y07604_at,Z19554_s_at,Z29064_at,Z69915_at,Z97074_at,Class
0,71,44,56,86,259,181,99,218,173,187,...,10,160,48,220,142,1026,28,81,52,?
1,348,165,234,626,129,397,36,623,265,124,...,124,112,133,132,67,2051,81,117,67,?
2,37,-4,25,106,350,-4,17,-1086,163,76,...,-38,-138,62,153,70,442,21,92,32,?
3,203,12,29,158,164,127,58,227,198,210,...,49,4,30,145,144,730,56,97,30,?
4,295,66,206,409,107,211,48,410,383,192,...,214,120,70,99,163,1694,151,80,59,?
5,82,20,30,129,277,56,41,211,174,96,...,15,14,47,205,140,1014,8,84,50,?
6,95,37,42,63,162,118,50,30,244,142,...,155,-12,69,33,178,3122,81,175,56,?
7,237,22,97,144,109,118,47,316,223,162,...,217,66,94,115,98,408,51,136,48,?
8,348,72,352,653,124,295,33,553,251,225,...,172,70,171,107,128,2456,214,47,45,?
9,224,278,168,237,136,580,53,524,217,178,...,78,179,166,128,168,1425,114,61,56,?


In [106]:
save_arff('./data/pp5i_test.bestN.arff',test_data.values,"default",test_data.columns)